In [1]:
class config():
    def __init__(self):
        self.vocab_size=65
        self.strvocab_size=3
        self.embedding_size=100
        self.strembedding_size=2
        self.feature_size=64
#         self.mi_kernel_size=[[1],[1,3],[1,3,5],[1,3,5,7]]
#         self.lnc_kernel_size=[[1],[1,3,5],[1,3,5,9],[1,3,5,9,15]]
#         self.mi_kernel_size=[2,4,6]
#         self.lnc_kernel_size=[4,8,16]
        self.mi_kernel_size=[1,3,5]
        self.lnc_kernel_size=[3,7,11]
        self.max_mi=25
        self.max_lnc=22743
        self.num_class=2

# In[2]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from models import *



In [2]:
import numpy as np
import time
import pandas as pd
import os
import argparse
import torch
import torch.nn as nn
from torch.optim import *
from sklearn.model_selection import KFold
from tqdm import tqdm 
from torch.utils.data import DataLoader,Dataset,ConcatDataset


from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score






# In[5]:


parser = argparse.ArgumentParser(description="type args")
parser.add_argument('-batch_size', type=int, default=64, help="BatchSize")
parser.add_argument('-epochs', type=int, default=100, help="Epoch Num")
parser.add_argument('-train', type=str, default='True', help="Train or Eval")
parser.add_argument('-cuda_id', type=int, default='0', help="CUDA Num")
parser.add_argument('-model_save_path', type=str,
                    default='./model', help="model save path")

args = parser.parse_args(args=[])

In [3]:
class Data(Dataset):
    def __init__(self,txtpath):
        if args.cuda_id != -1:
            device = 'cuda:{}'.format(args.cuda_id)
        else:
            device = 'cpu'

        self.data = txtpath
        
        self.featurelnc=pd.read_csv('../extracted_feature/lncfeature.csv',index_col=0)
        self.featuremi=pd.read_csv('../extracted_feature/mifeature.csv',index_col=0)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        lncid=torch.from_numpy(self.data[index,0]).cuda(device)
        miid=torch.from_numpy(self.data[index,1]).cuda(device)
        strlncid=torch.from_numpy(self.featurelnc.iloc[index].to_numpy()).cuda(device)
        strmiid=torch.from_numpy(self.featuremi.iloc[index].to_numpy()).cuda(device)
        label=self.data[index,4]
        return lncid,miid,strlncid.float(),strmiid.float(),label

In [4]:
raw=np.load('../Data/id_trainval_seqstr.npy',allow_pickle=True)
dataset = Data(raw)

In [5]:
torch.manual_seed(42)
test_ids=np.load('0_testset.npy')
test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
testloader = torch.utils.data.DataLoader(dataset,batch_size=1, sampler=test_subsampler)

In [6]:
def ACC(testloader,model1,model2,model3):
    softmax_0 = nn.Softmax(dim=1)
    correct1, total1 = 0, 0
    correct2, total2 = 0, 0
    correct3, total3 = 0, 0
    l=[]
    p1=[]
    s1=[]
   
    p2=[]
    s2=[]

    p3=[]
    s3=[]
    seq=[]
    with torch.no_grad():
      # Iterate over the test data and generate predictions
        for i, data in tqdm(enumerate(testloader, 0)):

            # Get inputs
            lncsams,misams,strlncsams,strmisams,labels = data
            seq.append(lncsams.detach().cpu())
            labels=labels.long()
            l.append([labels.detach().cpu()])
            # Generate outputs
            outputs1 = model1(misams,lncsams,strmisams,strlncsams)
            # Set total and correct
            _, predicted1 = torch.max(outputs1.data, 1)
            total1 += labels.size(0)
            correct1 += (predicted1 == labels.cuda(device)).sum().item()
            p1.append(predicted1.detach().cpu())
            s1.append([softmax_0(outputs1)[0][1].detach().cpu()])
            
            outputs2 = model2(misams,lncsams,strmisams,strlncsams)
            # Set total and correct
            _, predicted2 = torch.max(outputs2.data, 1)
            total2+= labels.size(0)
            correct2 += (predicted2 == labels.cuda(device)).sum().item()
            p2.append(predicted2.detach().cpu())
            s2.append([softmax_0(outputs2)[0][1].detach().cpu()])
            
            outputs3 = model3(misams,lncsams,strmisams,strlncsams)
            # Set total and correct
            _, predicted3 = torch.max(outputs3.data, 1)
            total3 += labels.size(0)
            correct3 += (predicted3 == labels.cuda(device)).sum().item()
            p3.append(predicted3.detach().cpu())
            s3.append([softmax_0(outputs3)[0][1].detach().cpu()])
            
            
    return (100.0 * correct1 / total1),(100.0 * correct2 / total2),(100.0 * correct3 / total3),l,seq,p1,s1,p2,s2,p3,s3
    

In [7]:
if args.cuda_id != -1:
    device = 'cuda:{}'.format(args.cuda_id)
else:
    device = 'cpu'
    
cf=config()
model1=AttentionMiRNATAR_v2(cf)
model1.load_state_dict(torch.load("./Results/fold1/model-epoch-90_v2.pth"))
model1.to(device)
model1.eval()
model2=AttentionMiRNATAR_v3(cf)
model2.load_state_dict(torch.load("./Results/fold1/model-epoch-90_v3.pth"))
model2.to(device)
model2.eval()
model3=AttentionMiRNATAR_v7(cf)
model3.load_state_dict(torch.load("./Results/fold1/model-epoch-90_v7.pth"))
model3.to(device)
model3.eval()

res1,res2,res3,l,seq,p1,s1,p2,s2,p3,s3=ACC(testloader,model1,model2,model3)


4968it [07:33, 10.94it/s]


In [8]:
print(res1,res2,res3)

90.05636070853463 90.09661835748793 86.47342995169082


In [9]:
from sklearn.metrics import confusion_matrix,roc_auc_score,f1_score,average_precision_score,roc_curve

# from collections import Counter
# for i in seq1[:10]:
#     c=0
#     for j in i[0]:
#         if j==0:
#             c+=1
#     print(22743-c)


w1=0.6
w2=0.2
w3=0.2

l=np.array(l).astype('float')
s=[s1[i][0]*w1+s2[i][0]*w2+s3[i][0]*w3 for i in range(len(s1))]
# s=[s1[i][0]*0.85+s2[i][0]*0.15 for i in range(len(s1))]


s1=np.array(s1).astype('float')
s2=np.array(s2).astype('float')
s3=np.array(s3).astype('float')

p_new=[]

FPR, recall, thresholds1 = roc_curve(l,s, pos_label=1)
gmean=np.sqrt(recall*(1-FPR))
index=np.argmax(gmean)
th1=thresholds1[index]
print(th1)

# FPR, recall, thresholds2 = roc_curve(l,s2, pos_label=1)
# gmean=np.sqrt(recall*(1-FPR))
# index=np.argmax(gmean)
# th2=thresholds2[index]

# FPR, recall, thresholds3 = roc_curve(l,s3, pos_label=1)
# gmean=np.sqrt(recall*(1-FPR))
# index=np.argmax(gmean)
# th3=thresholds3[index]

# print(th1,th2,th3)

# for i in range(len(s1)):
#     if s1[i]<th1:
#         c1=0
#     else:
#         c1=1
#     if s2[i]<th2:
#         c2=0
#     else:
#         c2=1
#     if s3[i]<th3:
#         c3=0
#     else:
#         c3=1

#     if c1+c2+c3>1:
#         p_new.append(1)
#     else:
#         p_new.append(0)

for i in s:
    if i<th1:
        p_new.append(0)
    else:
        p_new.append(1)
# s=s3

p=p_new

tn,fp,fn,tp = confusion_matrix(l, p).ravel() 
print(tn,fp,fn,tp)
pp = tp+fn
pn = tn+fp
sensitivity = tp / pp
specificity = tn / pn
precision = tp / (tp + fp)
acc = (tp+tn) / (pp+pn)
mcc = ((tp*tn)-(fp*fn))/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
print("F1-Score:{:.4f}".format(f1_score(l,p)))
print("AUC:{:.4f}".format(roc_auc_score(l,s)))
print("ACC:{:.4f}".format(acc))
print("Sn:{:.4f}".format(sensitivity))
print("Sp:{:.4f}".format(specificity))
print("Pre:{:.4f}".format(precision))
print("Mcc:{:.4f}".format(mcc))
print("PRC:{:.4f}".format(average_precision_score(l,s)))

/tmp/ipykernel_256/2195162950.py:16: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  l=np.array(l).astype('float')
/tmp/ipykernel_256/2195162950.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  l=np.array(l).astype('float')


0.78787875
2393 101 291 2183
F1-Score:0.9176
AUC:0.9678
ACC:0.9211
Sn:0.8824
Sp:0.9595
Pre:0.9558
Mcc:0.8446
PRC:0.9733


In [ ]:
fold4:
F1-Score:0.9242
AUC:0.9694
ACC:0.9261
Sn:0.8977
Sp:0.9548
Pre:0.9523
Mcc:0.8537
PRC:0.9725
    
fold2:   
0.7706207
2361 127 263 2217
F1-Score:0.9192
AUC:0.9694
ACC:0.9215
Sn:0.8940
Sp:0.9490
Pre:0.9458
Mcc:0.8442
PRC:0.9732

fold3
F1-Score:0.9136
AUC:0.9632
ACC:0.9147
Sn:0.8965
Sp:0.9331
Pre:0.9315
Mcc:0.8299
PRC:0.9674

fold5
F1-Score:0.9101
AUC:0.9656
ACC:0.9128
Sn:0.8867
Sp:0.9387
Pre:0.9348
Mcc:0.8267
PRC:0.9693

fold1
0.7949260383844375
2372 122 289 2185
F1-Score:0.9140
AUC:0.9656
ACC:0.9173
Sn:0.8832
Sp:0.9511
Pre:0.9471
Mcc:0.8364
PRC:0.9715

4968it [00:40, 123.59it/s]

89.87520128824477
